In [109]:
# Call in libraries
library(haven)
library(optimx)
library(foreign)
library(sandwich)
library(AER)
library(xtable)
library(texreg)
library(stargazer)

In [110]:
ps6_data <- read_dta("C:/Users/yafei/Dropbox/sponsor/data and program/ds_fac_underp.dta") 

In [ ]:
summary(ps6_data)

In [111]:
# keep only columns will use
ps6_df <- ps6_data[,c("spread_adj_par","borrower_lead_amt","amount", "term", "dum_public", "dum_sponsor", "dum_prepay", 
                      "dum_floor", "dum_refin", "dum_secure", "dum_cov", "rate_blr_sp", "primarypurpose", "sic2",
                      "loan_year", "companyid", "post")]

In [ ]:
summary(ps6_df)

In [112]:
# Calculate variables
ps6_df['abs_spread_adj'] <- abs(ps6_df['spread_adj_par'])
ps6_df['log_amt'] <- log(ps6_df['amount'])
ps6_df['log_term'] <- log(ps6_df['term'])
ps6_df$treat <- ifelse ((ps6_df$post == 1), 1, 0)


# Generate dummy variable for fixed effects
ps6_df$purpg <- factor(ps6_df$primarypurpose)
ps6_df$indg <- factor(ps6_df$sic2)
ps6_df$yearg <- factor(ps6_df$loan_year)
ps6_df$rateg <- factor(ps6_df$rate_blr_sp)
ps6_df$companyidg <- factor(ps6_df$companyid)

In [113]:
# OLS without fixed effects
adj_OLS <- lm(abs_spread_adj ~ borrower_lead_amt + log_amt + log_term + dum_public + 
              dum_sponsor + dum_prepay + dum_floor + dum_refin + dum_secure + dum_cov, data = ps6_df)

In [114]:
# OLS with fixed effects
adj_OLS_fix <- lm(abs_spread_adj ~ borrower_lead_amt + log_amt + log_term + dum_public + 
                  dum_sponsor + dum_prepay + dum_floor + dum_refin + dum_secure + dum_cov + 
                  purpg + indg + yearg + rateg + companyidg, data = ps6_df)
# Return robust standard errors
coeftest(adj_OLS_fix, vcov = vcovHC(adj_OLS_fix, "HC1"))


t test of coefficients:

                                                Estimate Std. Error t value
(Intercept)                                   -10.476399   7.460324 -1.4043
borrower_lead_amt                              -0.059836   0.011121 -5.3803
log_amt                                         1.938906   0.561398  3.4537
log_term                                        5.388674   2.406061  2.2396
dum_public                                      0.113334   1.167348  0.0971
dum_sponsor                                    -1.056510   1.164942 -0.9069
dum_prepay                                      9.793761   1.493660  6.5569
dum_floor                                       4.281884   3.006221  1.4243
dum_refin                                      -7.147771   1.346712 -5.3076
dum_secure                                      1.952803   2.777897  0.7030
dum_cov                                        -5.493836   1.315690 -4.1756
purpgCapital expend.                           -2.675869   6.2

In [119]:
summary(ps6_df$treat)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max.    NA's 
  0.000   1.000   1.000   0.784   1.000   1.000    4091 

In [118]:
# IV model
adj_IV <- ivreg(abs_spread_adj ~ borrower_lead_amt + log_amt + log_term + dum_public + 
                dum_sponsor + dum_prepay + dum_floor + dum_refin + dum_secure + dum_cov + 
                purpg + indg + yearg + rateg + companyidg |
                treat + log_amt + log_term + dum_public + dum_sponsor + dum_prepay + dum_floor + 
                dum_refin + dum_secure + dum_cov + purpg + indg + yearg + rateg + companyidg, data = ps6_df)
summary(adj_IV)


Call:
ivreg(formula = abs_spread_adj ~ borrower_lead_amt + log_amt + 
    log_term + dum_public + dum_sponsor + dum_prepay + dum_floor + 
    dum_refin + dum_secure + dum_cov + purpg + indg + yearg + 
    rateg + companyidg | treat + log_amt + log_term + dum_public + 
    dum_sponsor + dum_prepay + dum_floor + dum_refin + dum_secure + 
    dum_cov + purpg + indg + yearg + rateg + companyidg, data = ps6_df)

Residuals:
     Min       1Q   Median       3Q      Max 
-78.9531 -17.5119  -0.1264  14.4340 126.9850 

Coefficients:
                           Estimate Std. Error t value Pr(>|t|)    
(Intercept)                19.56674   31.15041   0.628 0.530172    
borrower_lead_amt           0.43167    0.97675   0.442 0.658703    
log_amt                     0.40268    2.00020   0.201 0.840524    
log_term                    7.24237    6.99865   1.035 0.301201    
dum_public                 -4.57261    4.19898  -1.089 0.276634    
dum_sponsor                -0.56280    6.44921  -0.087 0.93049

In [116]:
stargazer(adj_OLS, adj_OLS_fix, adj_IV, type = "text", title="Models",
                align=TRUE, column.labels=c("OLS without fixed effects","OLS with fixed effects", "2SLS IV"),
                dep.var.caption="", dep.var.labels="", dep.var.labels.include=False,
                covariate.labels=c('Intercept', 'Relationship', 'Log(Loan Amount)', 'Log(Loan Maturity)',
                                   'Dum_Public', 'Dum_Sponsor', 'Dum_prepay',
                                   'Dum_Floor', 'Dum_Refin', 'Dum_Secure', 'Dum_Cov'),
                no.space=TRUE, omit=c('purpg', 'indg', 'yearg', 'rateg', 'companyidg'), omit.yes.no=c("Yes", "No"),
                out = "models.txt")

ERROR: Error in .stargazer.wrap(..., type = type, title = title, style = style, : object 'False' not found


In [ ]:
print(stargazer(adj_OLS, adj_OLS_fix, adj_IV, type = "text", title="Models",
                align=TRUE, column.labels=c("OLS without fixed effects","OLS with fixed effects", "2SLS IV"),
                dep.var.caption="", dep.var.labels="", dep.var.labels.include=TRUE,
                covariate.labels=c('Intercept', 'Relationship', 'Log(Loan Amount)', 'Log(Loan Maturity)',
                                   'Dum_Public', 'Dum_Sponsor', 'Dum_prepay',
                                   'Dum_Floor', 'Dum_Refin', 'Dum_Secure', 'Dum_Cov'),
                no.space=TRUE, omit=c('purpg', 'indg', 'yearg', 'rateg', 'companyidg'), omit.yes.no=c("Yes", "No"),
                out = "models.txt"))
# to save to file just have keyworkd argument `out="path.filename.txt"`

In [ ]:
stargazer(adj_OLS, adj_OLS_fix, adj_IV, type = "text", title="Models",
                align=TRUE, column.labels=c("OLS without fixed effects","OLS with fixed effects", "2SLS IV"),
                dep.var.caption="", dep.var.labels="", dep.var.labels.include=TRUE,
                covariate.labels=c('Intercept', 'Relationship', 'Log(Loan Amount)', 'Log(Loan Maturity)',
                                   'Dum_Public', 'Dum_Sponsor', 'Dum_prepay',
                                   'Dum_Floor', 'Dum_Refin', 'Dum_Secure', 'Dum_Cov'),
                no.space=TRUE, omit=c('purpg', 'indg', 'yearg', 'rateg', 'companyidg'), omit.yes.no=c("Yes", "No"),
                out = "models.txt")
# to save to file just have keyworkd argument `out="path.filename.txt"`